We are fine-tuning a pre-trained BERT for sentiment analysis on imdb dataset.
Script written to run on any local system.

In this script, we register hooks for **all attention heads** in **all layers**.

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim

In [16]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

df = pd.read_csv('../../../../Vector Extraction/Transformer/IMDB Dataset.csv')
df = df[df['review'].apply(lambda x: len(tokenizer.tokenize(x)) >= 64)] # Filter out short reviews
df = df.reset_index(drop=True)
df['label'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [20]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):  # Adjust max_length as needed
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text using the provided tokenizer
        tokenized_text = self.tokenizer(
            text,
            padding='max_length',  # Pad to the specified max_length
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Extract relevant tensors
        input_ids = tokenized_text['input_ids'].squeeze()  # Remove the batch dimension
        attention_mask = tokenized_text['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}

In [21]:
texts = df['review']
labels = df['label']

In [22]:
# Tokenize and prepare the dataset
dataset = SentimentDataset(texts, labels, tokenizer)

# Fine-tuning the BERT model
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model):
        super(SentimentClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, 2)  # 768 is the size of BERT's hidden layers

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, 0, :]  # Use the [CLS] token representation
        logits = self.fc(pooled_output)
        return logits

# Initialize the sentiment classifier
classifier = SentimentClassifier(bert_model)

In [24]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(classifier.parameters(), lr=2e-5)

# Store Q, K, V vectors during training for all layers and attention heads
Q_vectors = []
K_vectors = []
V_vectors = []

def hook_fn(module, input, output):
    Q_vectors.append(module.query.weight.detach().cpu().numpy())
    K_vectors.append(module.key.weight.detach().cpu().numpy())
    V_vectors.append(module.value.weight.detach().cpu().numpy())

# Register hooks for all attention heads in all layers
for layer_idx in range(bert_model.config.num_hidden_layers):
    for head_idx in range(bert_model.config.num_attention_heads):
        attention_head = bert_model.encoder.layer[layer_idx].attention.self
        attention_head.register_forward_hook(hook_fn)

# Training loop
for epoch in range(5):
    for batch in DataLoader(dataset, batch_size=64, shuffle=True):
        optimizer.zero_grad()

        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        input_ids, attention_mask, labels = input_ids.to('cpu'), attention_mask.to('cpu'), labels.to('cpu')
        logits = classifier(input_ids, attention_mask)
        loss = criterion(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{5}, Batch Loss: {loss.item()}')

    print(f'Epoch {epoch + 1}/{5} completed')

Epoch 1/5, Batch Loss: 0.27735212445259094
Epoch 1/5, Batch Loss: 0.28619393706321716
Epoch 1/5, Batch Loss: 0.3369791507720947
Epoch 1/5, Batch Loss: 0.20712849497795105
Epoch 1/5, Batch Loss: 0.23315240442752838
Epoch 1/5, Batch Loss: 0.12093627452850342
Epoch 1/5, Batch Loss: 0.23601019382476807
Epoch 1/5, Batch Loss: 0.42484816908836365
Epoch 1/5, Batch Loss: 0.2970411479473114
Epoch 1/5, Batch Loss: 0.22186991572380066
Epoch 1/5, Batch Loss: 0.22488978505134583
Epoch 1/5, Batch Loss: 0.24221625924110413
Epoch 1/5, Batch Loss: 0.2795863747596741
Epoch 1/5, Batch Loss: 0.20949344336986542
Epoch 1/5, Batch Loss: 0.2770012319087982
Epoch 1/5, Batch Loss: 0.16266687214374542
Epoch 1/5, Batch Loss: 0.354194700717926
Epoch 1/5, Batch Loss: 0.2770816683769226
Epoch 1/5, Batch Loss: 0.180133655667305
Epoch 1/5, Batch Loss: 0.20969268679618835
Epoch 1/5, Batch Loss: 0.3412494659423828
Epoch 1/5, Batch Loss: 0.22317560017108917
Epoch 1/5, Batch Loss: 0.2403067797422409
Epoch 1/5, Batch Loss:

Epoch 1/5, Batch Loss: 0.4025864601135254
Epoch 1/5, Batch Loss: 0.24540719389915466
Epoch 1/5, Batch Loss: 0.24018698930740356
Epoch 1/5, Batch Loss: 0.23676158487796783
Epoch 1/5, Batch Loss: 0.22918540239334106
Epoch 1/5, Batch Loss: 0.2545163631439209
Epoch 1/5, Batch Loss: 0.18481995165348053
Epoch 1/5, Batch Loss: 0.3262762427330017
Epoch 1/5, Batch Loss: 0.24274101853370667
Epoch 1/5, Batch Loss: 0.34660229086875916
Epoch 1/5, Batch Loss: 0.23576468229293823
Epoch 1/5, Batch Loss: 0.24179324507713318
Epoch 1/5, Batch Loss: 0.254281222820282
Epoch 1/5, Batch Loss: 0.2374384105205536
Epoch 1/5, Batch Loss: 0.15218718349933624
Epoch 1/5, Batch Loss: 0.2263493686914444
Epoch 1/5, Batch Loss: 0.26467934250831604
Epoch 1/5, Batch Loss: 0.18467706441879272
Epoch 1/5, Batch Loss: 0.21687352657318115
Epoch 1/5, Batch Loss: 0.3255162239074707
Epoch 1/5, Batch Loss: 0.2575148344039917
Epoch 1/5, Batch Loss: 0.20473910868167877
Epoch 1/5, Batch Loss: 0.18480150401592255
Epoch 1/5, Batch Los

Epoch 1/5, Batch Loss: 0.22360867261886597
Epoch 1/5, Batch Loss: 0.2022029161453247
Epoch 1/5, Batch Loss: 0.1553092747926712
Epoch 1/5, Batch Loss: 0.27768170833587646
Epoch 1/5, Batch Loss: 0.12100352346897125
Epoch 1/5, Batch Loss: 0.2965785562992096
Epoch 1/5, Batch Loss: 0.21647711098194122
Epoch 1/5, Batch Loss: 0.20259153842926025
Epoch 1/5, Batch Loss: 0.27533572912216187
Epoch 1/5, Batch Loss: 0.17598985135555267
Epoch 1/5, Batch Loss: 0.16615775227546692
Epoch 1/5, Batch Loss: 0.2427208423614502
Epoch 1/5, Batch Loss: 0.21224308013916016
Epoch 1/5, Batch Loss: 0.2137722373008728
Epoch 1/5, Batch Loss: 0.18193432688713074
Epoch 1/5, Batch Loss: 0.13698074221611023
Epoch 1/5, Batch Loss: 0.24495857954025269
Epoch 1/5, Batch Loss: 0.12954893708229065
Epoch 1/5, Batch Loss: 0.12099589407444
Epoch 1/5, Batch Loss: 0.21748580038547516
Epoch 1/5, Batch Loss: 0.307110071182251
Epoch 1/5, Batch Loss: 0.23567846417427063
Epoch 1/5, Batch Loss: 0.21124309301376343
Epoch 1/5, Batch Loss

Epoch 1/5, Batch Loss: 0.17114636301994324
Epoch 1/5, Batch Loss: 0.13318276405334473
Epoch 1/5, Batch Loss: 0.17192858457565308
Epoch 1/5, Batch Loss: 0.38504865765571594
Epoch 1/5, Batch Loss: 0.21705284714698792
Epoch 1/5, Batch Loss: 0.15781269967556
Epoch 1/5, Batch Loss: 0.29857251048088074
Epoch 1/5, Batch Loss: 0.2837953567504883
Epoch 1/5, Batch Loss: 0.29985377192497253
Epoch 1/5, Batch Loss: 0.21231737732887268
Epoch 1/5, Batch Loss: 0.13664132356643677
Epoch 1/5, Batch Loss: 0.1420164853334427
Epoch 1/5, Batch Loss: 0.2744033932685852
Epoch 1/5, Batch Loss: 0.27651742100715637
Epoch 1/5, Batch Loss: 0.12997904419898987
Epoch 1/5, Batch Loss: 0.24684494733810425
Epoch 1/5, Batch Loss: 0.19424736499786377
Epoch 1/5, Batch Loss: 0.22828562557697296
Epoch 1/5, Batch Loss: 0.16889731585979462
Epoch 1/5, Batch Loss: 0.22855964303016663
Epoch 1/5, Batch Loss: 0.15971112251281738
Epoch 1/5, Batch Loss: 0.2093680500984192
Epoch 1/5, Batch Loss: 0.1724308729171753
Epoch 1/5, Batch Lo

Epoch 2/5, Batch Loss: 0.10076314955949783
Epoch 2/5, Batch Loss: 0.05797277018427849
Epoch 2/5, Batch Loss: 0.10415301471948624
Epoch 2/5, Batch Loss: 0.06889557838439941
Epoch 2/5, Batch Loss: 0.11086748540401459
Epoch 2/5, Batch Loss: 0.11154547333717346
Epoch 2/5, Batch Loss: 0.19700127840042114
Epoch 2/5, Batch Loss: 0.11055681109428406
Epoch 2/5, Batch Loss: 0.06238663196563721
Epoch 2/5, Batch Loss: 0.17876218259334564
Epoch 2/5, Batch Loss: 0.05644899979233742
Epoch 2/5, Batch Loss: 0.03525938838720322
Epoch 2/5, Batch Loss: 0.026104846969246864
Epoch 2/5, Batch Loss: 0.07720886915922165
Epoch 2/5, Batch Loss: 0.12176571786403656
Epoch 2/5, Batch Loss: 0.12159872055053711
Epoch 2/5, Batch Loss: 0.027354609221220016
Epoch 2/5, Batch Loss: 0.03301497548818588
Epoch 2/5, Batch Loss: 0.05192160606384277
Epoch 2/5, Batch Loss: 0.10203304886817932
Epoch 2/5, Batch Loss: 0.12229754030704498
Epoch 2/5, Batch Loss: 0.05622907355427742
Epoch 2/5, Batch Loss: 0.057808566838502884
Epoch 2/

...

Epoch 4/5, Batch Loss: 0.0014887847937643528
Epoch 4/5, Batch Loss: 0.0018587661907076836
Epoch 4/5, Batch Loss: 0.02263684570789337
Epoch 4/5, Batch Loss: 0.042566828429698944
Epoch 4/5, Batch Loss: 0.0014544667210429907
Epoch 4/5, Batch Loss: 0.002382439561188221
Epoch 4/5, Batch Loss: 0.0016038992907851934
Epoch 4/5, Batch Loss: 0.0028993498999625444
Epoch 4/5, Batch Loss: 0.00125764193944633
Epoch 4/5, Batch Loss: 0.004989138338714838
Epoch 4/5, Batch Loss: 0.07313980907201767
Epoch 4/5, Batch Loss: 0.039692580699920654
Epoch 4/5, Batch Loss: 0.052619900554418564
Epoch 4/5, Batch Loss: 0.003377358429133892
Epoch 4/5, Batch Loss: 0.010893828235566616
Epoch 4/5, Batch Loss: 0.0017812143778428435
Epoch 4/5, Batch Loss: 0.015269141644239426
Epoch 4/5, Batch Loss: 0.0007996163330972195
Epoch 4/5, Batch Loss: 0.00046341121196746826
Epoch 4/5, Batch Loss: 0.008022015914320946
Epoch 4/5, Batch Loss: 0.0016982612432911992
Epoch 4/5, Batch Loss: 0.002284630201756954
Epoch 4/5, Batch Loss: 0.

Epoch 5/5, Batch Loss: 0.001435240381397307
Epoch 5/5, Batch Loss: 0.0006003566086292267
Epoch 5/5, Batch Loss: 0.00036504140007309616
Epoch 5/5, Batch Loss: 0.0007149877492338419
Epoch 5/5, Batch Loss: 0.0006082868203520775
Epoch 5/5, Batch Loss: 0.0010206098668277264
Epoch 5/5, Batch Loss: 0.000528530334122479
Epoch 5/5, Batch Loss: 0.0004692111979238689
Epoch 5/5, Batch Loss: 0.0009733272599987686
Epoch 5/5, Batch Loss: 0.0006017856067046523
Epoch 5/5, Batch Loss: 0.0017979053081944585
Epoch 5/5, Batch Loss: 0.0034172215964645147
Epoch 5/5, Batch Loss: 0.0014494271017611027
Epoch 5/5, Batch Loss: 0.0015903688035905361
Epoch 5/5, Batch Loss: 0.0007201594999060035
Epoch 5/5, Batch Loss: 0.0014667020877823234
Epoch 5/5, Batch Loss: 0.014411203563213348
Epoch 5/5, Batch Loss: 0.00453649926930666
Epoch 5/5, Batch Loss: 0.0005380221409723163
Epoch 5/5, Batch Loss: 0.00184439227450639
Epoch 5/5, Batch Loss: 0.0020047943107783794
Epoch 5/5, Batch Loss: 0.0004446258826646954
Epoch 5/5, Batch

Epoch 5/5, Batch Loss: 0.001597503200173378
Epoch 5/5, Batch Loss: 0.0006616691243834794
Epoch 5/5, Batch Loss: 0.005244921892881393
Epoch 5/5, Batch Loss: 0.0001773944968590513
Epoch 5/5, Batch Loss: 0.10559281706809998
Epoch 5/5, Batch Loss: 0.09284800291061401
Epoch 5/5, Batch Loss: 0.0013318448327481747
Epoch 5/5, Batch Loss: 0.0007842360064387321
Epoch 5/5, Batch Loss: 0.0003226130793336779
Epoch 5/5, Batch Loss: 0.0015406180173158646
Epoch 5/5, Batch Loss: 0.00026075454661622643
Epoch 5/5, Batch Loss: 0.00865951832383871
Epoch 5/5, Batch Loss: 0.00014881441893521696
Epoch 5/5, Batch Loss: 0.0007211415213532746
Epoch 5/5, Batch Loss: 0.00038006645627319813
Epoch 5/5, Batch Loss: 0.00033255311427637935
Epoch 5/5, Batch Loss: 0.002221127273514867
Epoch 5/5, Batch Loss: 0.000283111265162006
Epoch 5/5, Batch Loss: 9.687295096227899e-05
Epoch 5/5, Batch Loss: 0.024064701050519943
Epoch 5/5, Batch Loss: 0.0011211769888177514
Epoch 5/5, Batch Loss: 0.00019378236902412027
Epoch 5/5, Batch

Epoch 5/5, Batch Loss: 0.003008250379934907
Epoch 5/5, Batch Loss: 0.03720518946647644
Epoch 5/5, Batch Loss: 0.00065519631607458
Epoch 5/5, Batch Loss: 0.0014511463232338428
Epoch 5/5, Batch Loss: 0.007323548663407564
Epoch 5/5, Batch Loss: 0.006776635535061359
Epoch 5/5, Batch Loss: 0.02847391739487648
Epoch 5/5, Batch Loss: 0.0012028669007122517
Epoch 5/5, Batch Loss: 0.002184818731620908
Epoch 5/5, Batch Loss: 0.009248916059732437
Epoch 5/5, Batch Loss: 0.002410248853266239
Epoch 5/5, Batch Loss: 0.04298490285873413
Epoch 5/5, Batch Loss: 0.0010526349069550633
Epoch 5/5, Batch Loss: 0.007524270564317703
Epoch 5/5, Batch Loss: 0.003316898364573717
Epoch 5/5, Batch Loss: 0.00448527280241251
Epoch 5/5, Batch Loss: 0.0013912832364439964
Epoch 5/5, Batch Loss: 0.007632564287632704
Epoch 5/5, Batch Loss: 0.00243652006611228
Epoch 5/5, Batch Loss: 0.0033635608851909637
Epoch 5/5, Batch Loss: 0.0016478791367262602
Epoch 5/5, Batch Loss: 0.0022829286754131317
Epoch 5/5, Batch Loss: 0.082842

Epoch 5/5, Batch Loss: 0.0008661941974423826
Epoch 5/5, Batch Loss: 0.01777811534702778
Epoch 5/5, Batch Loss: 0.02193978987634182
Epoch 5/5, Batch Loss: 0.023817231878638268
Epoch 5/5, Batch Loss: 0.0008740372722968459
Epoch 5/5, Batch Loss: 0.010309611447155476
Epoch 5/5, Batch Loss: 0.000690082146320492
Epoch 5/5, Batch Loss: 0.00029478143551386893
Epoch 5/5, Batch Loss: 0.00010524589742999524
Epoch 5/5, Batch Loss: 0.00026176797109656036
Epoch 5/5, Batch Loss: 0.0006528130616061389
Epoch 5/5, Batch Loss: 0.0013278041733428836
Epoch 5/5, Batch Loss: 0.00016419243183918297
Epoch 5/5, Batch Loss: 0.00011093715875176713
Epoch 5/5, Batch Loss: 0.006840005051344633
Epoch 5/5, Batch Loss: 0.026237202808260918
Epoch 5/5, Batch Loss: 0.0950041189789772
Epoch 5/5, Batch Loss: 0.0009442337905056775
Epoch 5/5, Batch Loss: 0.0004915312747471035
Epoch 5/5, Batch Loss: 0.0001237826800206676
Epoch 5/5, Batch Loss: 0.007544923108071089
Epoch 5/5, Batch Loss: 0.0018567857332527637
Epoch 5/5, Batch L

In [3]:
# len(Q_vectors)

In [26]:
len(Q_vectors[0])

768